In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, col, count, countDistinct, sum, avg
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType
from delta.tables import *
from delta import *
from delta import configure_spark_with_delta_pip


In [11]:
builder = SparkSession \
        .builder \
        .appName('healthcare_Aggregations')\
        .master('local')\
        .config("spark.driver.memory","2g")\
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")


spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [12]:
spark

In [13]:
source = spark.read \
                .format("delta").load("silver\\healthcare_trans")             

In [14]:
source.count()

26294

In [15]:
source.printSchema()

root
 |-- patients_id: string (nullable = true)
 |-- encounters_id: string (nullable = true)
 |-- FIRST: string (nullable = true)
 |-- LAST: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- BIRTHDATE: date (nullable = true)
 |-- DEATHDATE: date (nullable = true)
 |-- ENCOUNTERCLASS: string (nullable = true)
 |-- REASONCODE: long (nullable = true)
 |-- REASONDESCRIPTION: string (nullable = true)
 |-- BASE_COST: integer (nullable = true)
 |-- BASE_ENCOUNTER_COST: double (nullable = true)
 |-- TOTAL_CLAIM_COST: double (nullable = true)
 |-- PAYER: string (nullable = true)
 |-- PAYER_COVERAGE: double (nullable = true)
 |-- PAYER_NAME: string (nullable = true)
 |-- INGESTION_DATE: timestamp (nullable = true)
 |-- MODIFICATION_DATE: timestamp (nullable = true)
 |-- SOURCE: string (nullable = true)



In [16]:
source.createOrReplaceTempView("health")

In [ ]:
#Insurance Analysis
pop_insurance = spark.sql("SELECT CITY, ENCOUNTERCLASS, REASONCODE, REASONDESCRIPTION, BASE_COST, BASE_ENCOUNTER_COST,\
                        TOTAL_CLAIM_COST,PAYER_COVERAGE , PAYER_NAME\
                       FROM health LIMIT 3")
pop_insurance.show()

+-------+--------------+----------+-----------------+---------+-------------------+----------------+--------------+--------------------+
|   CITY|ENCOUNTERCLASS|REASONCODE|REASONDESCRIPTION|BASE_COST|BASE_ENCOUNTER_COST|TOTAL_CLAIM_COST|PAYER_COVERAGE|          PAYER_NAME|
+-------+--------------+----------+-----------------+---------+-------------------+----------------+--------------+--------------------+
| Boston|    ambulatory|  68496003|   Polyp of colon|     3737|              85.55|        10224.23|       6936.92|Blue Cross Blue S...|
|Chelsea|    outpatient|  68496003|   Polyp of colon|     6419|             142.58|          278.58|        152.69|Blue Cross Blue S...|
|   Hull|    outpatient|      NULL|             NULL|      747|              85.55|           85.55|           0.0|        NO_INSURANCE|
+-------+--------------+----------+-----------------+---------+-------------------+----------------+--------------+--------------------+



In [18]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x000001F016A90990>>